# IBM Applied Data Science Specialization - Capstone Project

## Open a Mexican Food(Oaxaca) Restaurant in Mexico City

### Norma Ruiz

# Table of contents
1. [Definition of the problem and objective](#introduction)
2. [Data](#paragraph1)
3. [Methodology](#paragraph2)
4. [Analysis](#paragraph3)  
5. [Results and Conclusion](#paragraph4)  

## Definition of the problem and objective

The approach of my project is based on the fact that I have one Mexican Restaurant(Oaxacan Food) in Mexico City and I like to open a second one in another neighbourhood of Mexico City.
I want to use the techniques learned in the course to find neighbourhoods in Mexico City which can be good candidates to place the new restaurant.

Oaxaca is one of the main gastronomic centers of Mexico whose cuisine is internationally known.

I will collect information of the neighbourhoods in Mexico City which has restaurants of different types of food but not Mexican Food.

## Data

### This is the data used to solve the problem 

In [ ]:
# -*- coding: utf-8 -*-

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import io
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
! pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [3]:
CLIENT_ID = 'CW2HOTRKSJE2UU0D4KAOXX1WOE3E2BNTE0EQI0SUEVDROLOH'
CLIENT_SECRET = 'FYUP3CWSXYUK0SHQWSR04G130M51X551SANCCKPKEZSDRUXU'
VERSION = '20180605' # Foursquare API version
print('Your credentials: ')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials: 
CLIENT_ID: CW2HOTRKSJE2UU0D4KAOXX1WOE3E2BNTE0EQI0SUEVDROLOH
CLIENT_SECRET: FYUP3CWSXYUK0SHQWSR04G130M51X551SANCCKPKEZSDRUXU


In [4]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    return (nearby_venues)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

### For my actual Restaurant

I have a Mexican Restaurant in a neighbourhood in Mexico City called "Credito Constructor".
This neighbourhood is located inside "Benito Juarez" district.
First I will use Nominatim from geopy.geocoders library to get the latitude and longitude 
for this neighbourhood.

In [5]:
my_rest = 'Febo 48, Crédito Constructor, 03940 Ciudad de México, CDMX, México'
myrest_barrio = "My Restaurant"
address = my_rest 

In [6]:
geolocator = Nominatim(user_agent="myrest_explorer")
location = geolocator.geocode(address)
myrest_latitud = location.latitude
myrest_longitud = location.longitude
print(location, myrest_latitud, myrest_longitud)

Febo, Crédito constructor, Benito Juárez, CDMX, 03940, México 19.3658436 -99.1815044


### Get the venues around my Restaurant

Then I will use the Foursquare API with the explore endpoint to get recommended venues 
near my restaurant current location. 
I will limit the search to a radius of 1000 meters and 100 venues at most.

In [7]:
radius = 1000
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, myrest_latitud, myrest_longitud, VERSION, radius, LIMIT)
print(url)

results = requests.get(url).json()
print('There are {} venues around your location.'.format(len(results['response']['groups'][0]['items'])))

myrest_venues = results['response']['groups'][0]['items']

myrest_nearby_venues = json_normalize(myrest_venues)  # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng', 'venue.categories']
myrest_nearby_venues = myrest_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
myrest_nearby_venues['venue.categories'] = myrest_nearby_venues.apply(get_category_type, axis=1)

# clean columns
myrest_nearby_venues.columns = [col.split(".")[-1] for col in myrest_nearby_venues.columns]

myrest_nearby_venues['Neighborhood'] = myrest_barrio
myrest_nearby_venues['Neighborhood Latitude'] = myrest_latitud
myrest_nearby_venues['Neighborhood Longtude'] = myrest_longitud
cols = myrest_nearby_venues.columns.tolist()
cols = cols[-3:] + cols[:-3]
myrest_nearby_venues = myrest_nearby_venues[cols]

myrest_nearby_venues.head()
print('{} venues were returned by Foursquare.'.format(myrest_nearby_venues.shape[0]))
print('There are {} uniques categories.'.format(len(myrest_nearby_venues['categories'].unique())))


https://api.foursquare.com/v2/venues/explore?client_id=CW2HOTRKSJE2UU0D4KAOXX1WOE3E2BNTE0EQI0SUEVDROLOH&client_secret=FYUP3CWSXYUK0SHQWSR04G130M51X551SANCCKPKEZSDRUXU&ll=19.3658436,-99.1815044&v=20180605&radius=1000&limit=100
There are 100 venues around your location.
100 venues were returned by Foursquare.
There are 59 uniques categories.


### Finding the location for the new restaurant

To get data from Mexico City neighbourhoods I will use data from https://download.geonames.org/export/zip
Specifically I will download a MX.zip file.

The file is a list of Districts and Neighbourhoods in Mexico City. I prefer to work with 4 of 16 districts in Mexico City. They are: Benito Juarez, Coyoacan, Cuauhtemoc y Miguel Hidalgo. These districts have a great variety of restaurants and it is relatively easy to get there because they are well known and they are very well located.

In [8]:
# Obtenido de http://download.geonames.org/export/zip/
url = "https://www.dropbox.com/s/4wg9dgr478xrq2o/delegacion_colonia_sa.csv?dl=1"
urlData = requests.get(url).content
barrios_df = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
barrios_df.shape

(377, 2)

In [9]:
barrios_df.head()

,Delegacion,Colonia
0,Coyoacan,Villa Coyoacan
1,Coyoacan,Delegacion Politica Coyoacan
2,Coyoacan,Barrio Santa Catarina
3,Coyoacan,Barrio La Concepcion
4,Coyoacan,Barrio San Lucas


Now I get the coordinates for each neighbourhood.

In [10]:
geo_barrios = pd.DataFrame(columns=['Delegacion', 'Colonia', 'Latitud', 'Longitud'])
j = 0

for idx, row in barrios_df.iterrows():
    address = row['Colonia'] + ', ' + row['Delegacion'] +  ', Distrito Federal'
    geolocator = Nominatim(user_agent="cdmx_explorer")
    location = geolocator.geocode(address, timeout=30)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
        geo_barrios.loc[j] = [row['Delegacion'], row['Colonia'], latitude, longitude]
        j += 1

print(geo_barrios.shape)
print(geo_barrios.head())

(257, 4)
  Delegacion               Colonia    Latitud   Longitud
0   Coyoacan        Villa Coyoacan  19.350175 -99.162254
1   Coyoacan  Barrio La Concepcion  19.344669 -99.157403
2   Coyoacan      Barrio San Lucas  19.315626 -99.132968
3   Coyoacan     Parque San Andres  19.345394 -99.148279
4   Coyoacan            Del Carmen  19.354290 -99.161797


I will use the Foursquare API with the explore endpoint to get the recommended venues for each neighbourhood (using radius of 1000 meters and limit of 100 venues per neighbourhood).

In [12]:
radius = 1000
LIMIT = 100
cdmx_venues = getNearbyVenues(names=geo_barrios['Colonia'],
                              latitudes=geo_barrios['Latitud'],
                              longitudes=geo_barrios['Longitud']
                             )
print(cdmx_venues.shape)
print(cdmx_venues.head())
print('There are {} unique categories.'.format(len(cdmx_venues['Venue Category'].unique())))

Villa Coyoacan
Barrio La Concepcion
Barrio San Lucas
Parque San Andres
Del Carmen
Viveros de Coyoacan
San Diego Churubusco
Campestre Churubusco
Prado Churubusco
Ermita Churubusco
Hermosillo
Paseos de Taxquena
Taxquena
Ajusco
Ajusco Montserrat
Pedregal de Monserrat
Romero de Terreros
ISSSTE
Barrio Oxtopulco Universidad
Torres de Chimalistac
Fortin de Chimalistac
Romero de Terreros
Villa de San Francisco
Pedregal de Coyoacan
Pueblo de los Reyes
Barrio del Nino Jesus
El Rosedal
Pacifico
Rosedal
Rinconada de los Reyes
Zapata
Montserrat
Mariana
Copilco El Bajo
Copilco Universidad ISSSTE
Altillo Universidad
Copilco
Acasulco
Integracion Latinoamericana
Empleados Federales
Copilco Universidad
Copilco El Alto
Residencial la Cantera
Pedregal de Santo Domingo
Atlantida
Ciudad Jardin
Xotepingo 101
Pueblo La Candelaria
El Rosario
Huayamilpas
Nueva Diaz Ordaz
Educacion
Petrolera Taxquena
Ex-Ejido de San Francisco Culhuacan
Culhuacan CTM Seccion V
Culhuacan CTM Seccion II
Culhuacan CTM Seccion I
El C

## Methodology

In order to use KMEANS we have to transform the data so that they are all numeric. 
For this purpose I will follow the following steps:
* Put together the location data of CDMX Districts and Neighbourhoods
* Collect the data of places of interest of CDMX 
* Use "onehot encoding" to transpose the categories of the places of interest 
and convert them to numerical values
* Group the resulting matrix by neighbourhood, using the average value of each category
* Apply KMEANS using 3 clusters

In [13]:
# filtrar colonias con 5 ó más restaurantes
cdmx_venues.columns = cdmx_venues.columns.str.replace(' ', '_')
sql_stmt = "select * " \
           "from cdmx_venues " \
           "where Venue_Category like '%Restaurant%' " \
           "group by Neighborhood " \
           "having count(*) >= 5;"
cdmx_rest = pysqldf(sql_stmt)
print(cdmx_rest.shape)

(179, 7)


In [14]:
cdmx_rest.head()

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,16 de Septiembre,19.402618,-99.205638,Potzollcalli,19.400858,-99.207978,Mexican Restaurant
1,5 de Mayo,19.448651,-99.199985,Panza Pancho,19.452669,-99.201216,Restaurant
2,8 de Agosto,19.392489,-99.191089,sushi makin japonés,19.389690,-99.192355,Sushi Restaurant
3,Acacias,19.363974,-99.174171,Taquearte,19.364231,-99.178868,Mexican Restaurant
4,Acasulco,19.342915,-99.180741,Puntarena,19.344686,-99.179467,Seafood Restaurant


In [15]:
cdmx_rest.tail()

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
174,Xotepingo,19.328499,-99.140857,EL PERLAZUL,19.327581,-99.136700,Seafood Restaurant
175,Xotepingo 101,19.328499,-99.140857,EL PERLAZUL,19.327581,-99.136700,Seafood Restaurant
176,Zacahuitzco,19.373199,-99.138781,Los Sopes de la Nueve,19.369390,-99.136962,Mexican Restaurant
177,Zapata,19.370670,-99.164806,Le Pain Quotidien,19.374597,-99.162623,French Restaurant
178,alamos,19.398194,-99.141743,Nasty Zombie,19.396131,-99.137627,Theme Restaurant


In [16]:
# de esas colonias seleccionar las que tengan menos de 2 restaurantes mexicanos
sql_stmt = "select * " \
           "from cdmx_rest " \
           "where Venue_Category = 'Mexican Restaurant' " \
           "group by Neighborhood " \
           "having count(*) <= 2;"
cdmx_mex_rest = pysqldf(sql_stmt)
print(cdmx_mex_rest.shape)

(84, 7)


In [17]:
# filtrar la lista original "cdmx_venues" dejando sólo las colonias que estan en el dataframe "cdmx_mex_rest"
print(cdmx_venues.shape)
sql_stmt = "select v.* " \
           "from cdmx_venues as v inner join cdmx_mex_rest as m " \
           "on v.Neighborhood = m.Neighborhood " \
           "order by v.Neighborhood,  v.Venue_Category, v.Venue;"
cdmx_candidates = pysqldf(sql_stmt)
print(cdmx_candidates.shape)

(11233, 7)
(3955, 7)


In [18]:
cols = cdmx_candidates.columns.tolist()
myrest_nearby_venues.columns = cols

In [19]:
cdmx_candidates = cdmx_candidates.append(myrest_nearby_venues, ignore_index=True)

In [20]:
print(cdmx_candidates.shape)
print(cdmx_candidates.head())

(4055, 7)
       Neighborhood  Neighborhood_Latitude  Neighborhood_Longitude  \
0  16 de Septiembre              19.402618              -99.205638   
1  16 de Septiembre              19.402618              -99.205638   
2  16 de Septiembre              19.402618              -99.205638   
3  16 de Septiembre              19.402618              -99.205638   
4  16 de Septiembre              19.402618              -99.205638   

                    Venue  Venue_Latitude  Venue_Longitude     Venue_Category  
0    LeCaroz Observatorio       19.401540       -99.202595             Bakery  
1  Cantina La Importadora       19.403918       -99.205370                Bar  
2  Garabatos Hospital ABC       19.400958       -99.204228        Coffee Shop  
3               Starbucks       19.399879       -99.204226        Coffee Shop  
4                    Oxxo       19.400796       -99.208221  Convenience Store  


In [87]:
cdmx_candidates.groupby('Neighborhood').count()

,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Neighborhood,,,,,,
16 de Septiembre,21,21,21,21,21,21
Acacias,17,17,17,17,17,17
Albert,40,40,40,40,40,40
America,30,30,30,30,30,30
Ampliacion Daniel Garza,29,29,29,29,29,29
Ampliacion Popo,31,31,31,31,31,31
Argentina Poniente,63,63,63,63,63,63
Atlampa,24,24,24,24,24,24
Barrio La Concepcion,49,49,49,49,49,49


In [21]:
print('There are {} uniques categories.'.format(len(cdmx_candidates['Venue_Category'].unique())))

There are 272 uniques categories.


In [22]:
# one hot encoding
cdmx_onehot = pd.get_dummies(cdmx_candidates[['Venue_Category']], prefix="", prefix_sep="")

In [23]:
# add neighbourhood column back to dataframe
cdmx_onehot['Nborhood'] = cdmx_candidates['Neighborhood']

In [24]:
# move neighbourhood column to the first column
fixed_columns = [cdmx_onehot.columns[-1]] + list(cdmx_onehot.columns[:-1])
cdmx_onehot = cdmx_onehot[fixed_columns]

In [25]:
print(cdmx_onehot.shape)
print(cdmx_onehot.head())


(4055, 273)
           Nborhood  Accessories Store  American Restaurant  Antique Shop  \
0  16 de Septiembre                  0                    0             0   
1  16 de Septiembre                  0                    0             0   
2  16 de Septiembre                  0                    0             0   
3  16 de Septiembre                  0                    0             0   
4  16 de Septiembre                  0                    0             0   

   Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0                 0                       0            0           0   
1                 0                       0            0           0   
2                 0                       0            0           0   
3                 0                       0            0           0   
4                 0                       0            0           0   

   Arts & Crafts Store  Asian Restaurant ...   Warehouse Store  Watch Shop  \
0             

In [26]:
cdmx_grouped = cdmx_onehot.groupby('Nborhood').mean().reset_index()
print(cdmx_grouped.shape)
print(cdmx_grouped.head())

(85, 273)
                  Nborhood  Accessories Store  American Restaurant  \
0         16 de Septiembre                0.0                  0.0   
1                  Acacias                0.0                  0.0   
2                   Albert                0.0                  0.0   
3                  America                0.0                  0.0   
4  Ampliacion Daniel Garza                0.0                  0.0   

   Antique Shop  Arepa Restaurant  Argentinian Restaurant  Art Gallery  \
0           0.0               0.0                     0.0          0.0   
1           0.0               0.0                     0.0          0.0   
2           0.0               0.0                     0.0          0.0   
3           0.0               0.0                     0.0          0.0   
4           0.0               0.0                     0.0          0.0   

   Art Museum  Arts & Crafts Store  Asian Restaurant ...   Warehouse Store  \
0         0.0                  0.0            

### Analysis

For each neighbourhood I will look for what are the 10 most frequent venue categories. 
This will help me to obtain the conclusions about the neighbourhoods candidates 
that are more similar to my actual restaurant neighbourhood in Mexico City.

In [27]:
num_top_venues = 10

In [28]:
indicators = ['st', 'nd', 'rd']

In [29]:
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [30]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cdmx_grouped['Nborhood']

In [31]:
for ind in np.arange(cdmx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cdmx_grouped.iloc[ind, :], num_top_venues)

In [34]:
print(neighborhoods_venues_sorted.shape)

(85, 11)


In [36]:
neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,16 de Septiembre,Mexican Restaurant,Taco Place,Coffee Shop,Seafood Restaurant,Pharmacy,Convenience Store,Snack Place,Speakeasy,Bar,Bakery
1,Acacias,Mexican Restaurant,Coffee Shop,Bowling Alley,Bakery,Bistro,Seafood Restaurant,Spa,Gym / Fitness Center,Gym,Soccer Field
2,Albert,Mexican Restaurant,Taco Place,Pizza Place,Breakfast Spot,Dessert Shop,Japanese Restaurant,Lounge,Sandwich Place,Food Truck,Coffee Shop
3,America,Mexican Restaurant,Taco Place,Pharmacy,Furniture / Home Store,Ice Cream Shop,Food Truck,Restaurant,Convenience Store,Bakery,Skate Park
4,Ampliacion Daniel Garza,Taco Place,Museum,Mexican Restaurant,Burger Joint,Food Truck,Racetrack,Public Art,Performing Arts Venue,College Academic Building,Salon / Barbershop
5,Ampliacion Popo,Mexican Restaurant,Residential Building (Apartment / Condo),Steakhouse,Taco Place,Park,Sporting Goods Shop,Snack Place,Spanish Restaurant,Food & Drink Shop,Sushi Restaurant
6,Argentina Poniente,Mexican Restaurant,Taco Place,Coffee Shop,Pet Store,Bakery,Deli / Bodega,Sushi Restaurant,Restaurant,Seafood Restaurant,Café
7,Atlampa,Mexican Restaurant,Bakery,Brewery,Auto Garage,Optical Shop,Performing Arts Venue,Bar,Nightclub,Food Truck,Pharmacy
8,Barrio La Concepcion,Mexican Restaurant,Pizza Place,Coffee Shop,Convenience Store,Food Truck,Donut Shop,Italian Restaurant,Café,Public Art,Nail Salon
9,Barrio Oxtopulco Universidad,Clothing Store,Sporting Goods Shop,Bookstore,Ice Cream Shop,Shopping Mall,Mexican Restaurant,Café,Boutique,Japanese Restaurant,Coffee Shop


### To find similar neighbourdoods

I'm going to use KMEANS to group neighbourhoods in Mexico City to see which ones are more similar to my actual restaurant neighbourhood.
I will try with several number of clusters to find the best value for K. For this purpose I will use 2 metrics:
* Silhouette Coefficient which bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters. The score is higher when clusters are dense and well separated.
* Calinski-Harabaz index which is higher when clusters are dense and well separated.

In [37]:
cdmx_grouped_clustering = cdmx_grouped.drop('Nborhood', 1)
best_sc = 0
best_sc_k = 0
best_chi = 0 
best_chi_k = 0

In [38]:
for x in range(3,12):
    kclusters = x
    # run k-means clustering
    kmeans_model = KMeans(n_clusters=kclusters, random_state=0).fit(cdmx_grouped_clustering)
    # get cluster labels
    labels = kmeans_model.labels_
    # compute Silhouette Coefficient
    sc = metrics.silhouette_score(cdmx_grouped_clustering, labels, metric='euclidean')
    print('Number of clusters: ', kclusters, ' Silhouette Coefficient: ', sc)
    # compute Calinski-Harabaz Index
    chi = metrics.calinski_harabaz_score(cdmx_grouped_clustering, labels)
    print('Number of clusters: ', kclusters, ' Calinski-Harabaz Index: ', chi)
    if sc > best_sc:
        best_sc = sc
        best_sc_k = kclusters
    if chi > best_chi:
        best_chi = chi
        best_chi_k = kclusters

Number of clusters:  3  Silhouette Coefficient:  0.14907923636
Number of clusters:  3  Calinski-Harabaz Index:  12.7788194437
Number of clusters:  4  Silhouette Coefficient:  0.112077701359
Number of clusters:  4  Calinski-Harabaz Index:  10.4149768069
Number of clusters:  5  Silhouette Coefficient:  0.124551448834
Number of clusters:  5  Calinski-Harabaz Index:  8.59389607432
Number of clusters:  6  Silhouette Coefficient:  0.101414832064
Number of clusters:  6  Calinski-Harabaz Index:  7.76676618463
Number of clusters:  7  Silhouette Coefficient:  0.105604112231
Number of clusters:  7  Calinski-Harabaz Index:  7.18803594163
Number of clusters:  8  Silhouette Coefficient:  0.117355947726
Number of clusters:  8  Calinski-Harabaz Index:  6.59347174814
Number of clusters:  9  Silhouette Coefficient:  0.100319480923
Number of clusters:  9  Calinski-Harabaz Index:  5.83418564867
Number of clusters:  10  Silhouette Coefficient:  0.0980862752504
Number of clusters:  10  Calinski-Harabaz Inde

In [39]:
print('Best # clusters according to Silhouette Coefficient: ', best_sc_k, ' with score = ', best_sc)
print('Best # clusters according to Calinski-Harabaz Index: ', best_chi_k, ' with score = ', best_chi)

Best # clusters according to Silhouette Coefficient:  3  with score =  0.14907923636
Best # clusters according to Calinski-Harabaz Index:  3  with score =  12.7788194437


I choose K from the best Silhouette Coefficient to use for the final model.


I'm going to calculate the distance of each point to its corresponding centroid 
using kmeans.transform.

In [40]:
# set number of clusters
kclusters = best_sc_k

In [41]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cdmx_grouped_clustering)
# get distance from centroids
distance = kmeans.transform(cdmx_grouped_clustering)

Add clustering labels and find cluster number and index value for neighbourhoods in Mexico City.

In [43]:
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 0, 2, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0,
       2, 2, 1, 1, 0, 2, 0, 0, 1, 0, 2, 1, 1, 1, 0, 0, 2, 2, 0, 1, 2, 1, 2,
       2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 1, 2, 0, 0, 0, 2], dtype=int32)

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# find index of "My restaurant"
myrest_ind = neighborhoods_venues_sorted.index[neighborhoods_venues_sorted['Neighborhood'] == myrest_barrio].tolist()
# find cluster of "My restaurant"
myrest_cluster = neighborhoods_venues_sorted.loc[myrest_ind]['Cluster Labels'].values[0]
# create dataframe with centroids distance to "My restaurant" cluster
dist_df = pd.DataFrame(distance[:,myrest_cluster], columns=['distance'])
dist_df['cluster'] = kmeans.labels_
# sort dataframe by cluster and distance
dist_df.sort_values(by=['cluster', 'distance'], inplace=True)
dist_df.reset_index(inplace=True)
# keep only the rows from the "My restaurant" cluster
dist_df = dist_df.loc[dist_df['cluster'] == myrest_cluster]
# get "My restaurant" position in sorted dataframe
myrest_pos = dist_df.index[dist_df['index'] == myrest_ind].tolist()[0]
# get 5 closest neighbourhoods from CDMX neighbourhood
top5 = dist_df.loc[myrest_pos+1:myrest_pos+5]
top5 = top5.append(dist_df.loc[myrest_pos-5:myrest_pos], ignore_index=True)
neighborhoods_venues_sorted = neighborhoods_venues_sorted.join(top5.set_index('index'), how='inner')

Join neighbourhoods_venues_sorted dataframe to geo_barrios dataframe to get the final result.

In [45]:
# agregar latitud y longitud a neighbourhoods_venues_sorted
myrest_row = {"Delegacion" : "Benito Juarez", "Colonia" : myrest_barrio, 
              "Latitud" : myrest_latitud, "Longitud" : myrest_longitud}
geo_barrios = geo_barrios.append(myrest_row, ignore_index=True)

In [46]:
sql_stmt = "select n.*, b.Latitud, b.Longitud " \
           "from neighborhoods_venues_sorted as n inner join geo_barrios as b " \
           "on n.Neighborhood = b.Colonia " \
           "order by n.Neighborhood;"
cdmx_merged = pysqldf(sql_stmt)

In [47]:
cdmx_merged.loc[cdmx_merged['Neighborhood'] == myrest_barrio, 'cluster'] = -1
cdmx_merged

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,distance,cluster,Latitud,Longitud
0,0,Argentina Poniente,Mexican Restaurant,Taco Place,Coffee Shop,Pet Store,Bakery,Deli / Bodega,Sushi Restaurant,Restaurant,Seafood Restaurant,Café,0.111957,0,19.393792,-99.162197
1,0,Centro Urbano Benito Juarez,Taco Place,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Pizza Place,Burrito Place,Restaurant,Italian Restaurant,Creperie,Flea Market,0.091958,0,19.409569,-99.157480
2,0,Letran Valle,Mexican Restaurant,Taco Place,Café,Coffee Shop,Food Truck,Ice Cream Shop,Pizza Place,Bakery,Pharmacy,Restaurant,0.118077,0,19.376082,-99.155022
3,0,Mixcoac,Taco Place,Mexican Restaurant,Coffee Shop,Cupcake Shop,Venezuelan Restaurant,Food Truck,Bar,Rock Climbing Spot,Residential Building (Apartment / Condo),Restaurant,0.120771,0,19.376019,-99.186967
4,0,My Restaurant,Coffee Shop,Mexican Restaurant,Seafood Restaurant,Restaurant,Ice Cream Shop,Cosmetics Shop,Argentinian Restaurant,Steakhouse,Pizza Place,Taco Place,0.097487,-1,19.365844,-99.181504
5,0,Procuraduria General de La Republica,Coffee Shop,Mexican Restaurant,Italian Restaurant,Hotel,Bakery,Boutique,Pizza Place,Salad Place,Cosmetics Shop,Café,0.114632,0,19.430282,-99.162172
6,0,San Juan,Mexican Restaurant,Coffee Shop,Venezuelan Restaurant,Bakery,Restaurant,Taco Place,Hotel,Food Truck,Steakhouse,Sporting Goods Shop,0.116039,0,19.378014,-99.184704


Draw a map of the selected neighbourhoods in Mexico City.

In [48]:
# create map
map_clusters = folium.Map(location=[myrest_latitud, myrest_longitud], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow_r(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cdmx_merged['Latitud'], cdmx_merged['Longitud'], cdmx_merged['Neighborhood'],
                                  cdmx_merged['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)

#display map
map_clusters    

## Results and Conclusion

The result of this exercise shows that it is possible to help people who are in a situation similar 
to the one described in this case, that is, someone who is looking for a place to open a second 
restaurant of the same type of food and wants to find conditions similar to those in the current 
restaurant using public data available through the Foursquare API.

In KMEANS one of the difficulties is the choice of the value for K. 
To decide what value to use, I executed the algorithm with different K values and, for each case, 
I calculated the Silhouette Coefficient and the Calinski-Harabaz Index. 
These are 2 metric that allow us to decide if we obtain dense and well separated clusters. 
With both indicators the best value for K was 3.

The venues of my restaurant neighborhood in Mexico City were added to the venues of 
the 7 neighborhoods in other districts and I generated 3 clusters. 
I searched within the cluster where my neighborhood in Mexico City was assigned (cluster 0), 
for those neighborhoods in Mexico City that were closest considering the Euclidean distance of 
each one of them to the cluster centroid.

The characteristics that distinguish my actual restaurant neighbourhood, 
according to the results of Foursquare, is the diversity of places to eat, shops and 
places to exercise. These same characteristics are present in almost all the selected neighbourhoods.

The technique used to select the candidate neighbourhoods is illustrated in the following figure:

<img src="https://www.dropbox.com/s/ky4khcw3ec7kooi/clustergraph.png?dl=1",width=600px,height=600px>

Of course, the final decision can not be based solely on the results of this analysis. 
Rather it should be considered as a tool to narrow the options that must be investigated in 
greater detail.
For example, one way to enrich the results of the analysis would be by adding demographic 
and socioeconomic attributes to each of the neighbourhoods. 
This would result in more similar and homogeneous clusters.